### 2020-04-01 09:25:17  같은거 중복으로 들어오는 오류발생
* 추정 page넘버 돌아가는거 미작동

In [1]:
# # 선행설치
# !pip install beautifulsoup4
# !pip install requests
# !pip install lxm
# # 연관분석
# !pip install apyori
# # 자연어처리
# !pip install konlpy (JDK 설치가 되어있어야함 )

In [1]:
import re
import json
import math
import datetime
import pandas as pd

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup #크롤링

import konlpy
from konlpy.tag import Komoran #자연어처리
komoran = Komoran(userdic='./user_dictionary.txt')

from apyori import apriori #연관분석
from tqdm import tqdm_notebook #진행과정 시각화 

In [2]:
#페이지 카운트 수로 api호출된다
#1,000개일때 3분 
#100*100개 만개의 경우 "비정상적인 트래픽을 감지"해서 오류걸림 
naver_client_id = "jXD6KJvReVAPrEVOluPD"
naver_client_secret = "xDUhz1vSpU"
keyword = '양양'
display_count = 1
page_count = 1000
sort_type = 'sim'

In [3]:
def del_outword(string):
    #이모지제거
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')
    
    string = emoji_pattern.sub(r'',string)
    string = han.sub(r'',string)
    
    return string

In [4]:
def get_blog_post(keyword, display_count, page_count, sort_type,save = True):
    '''
    keyword : 검색하고 싶은 키워드
    display_count : 한 페이지당 표출할 개수 min = 10, max = 100
    page_count : 총 진행할 페이지의 수 min = 1, max = 1000
    sort_type : 정렬옵션 "sim" (유사도순), "date" (날짜순)
    ''' 
    #날짜와 본문내용 저장
    postdates = []
    strings = []
    urls = []
    titles = []
    blogger_names = []
    
    encode_keyword = urllib.parse.quote(keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 페이지수까지 까지 각각 하나씩 긁어옴 
    for i in tqdm_notebook(range(1, page_count + 1),desc = "page work"):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()
         
        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))
        #items의 개수만큼씩 진행
            for j in range(0, len(response_body_dict['items'])):
                blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")
                urls.append(blog_post_url)
                #블로그 url안에 들어가기(아직 크롤링불가)
                get_blog_post_content_code = requests.get(blog_post_url)
                get_blog_post_content_text = get_blog_post_content_code.text
                get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
                #크롤링가능한 url에 접속
                real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                get_real_blog_post_content_code = requests.get(real_blog_post_url)
                get_real_blog_post_content_text = get_real_blog_post_content_code.text
                get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')
                #본문부분 추출 
                blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                if len(blog_post_content) == 0:
                    blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

                #전체 텍스트 
                string = ""
                for sentence in blog_post_content[0].stripped_strings:
                    string += " "+sentence.replace('\xa0'," ")
                #비언어 텍스트제거 
                string = del_outword(string)
                strings.append([string]) 

                #포스트날짜
                postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],"%Y%m%d").strftime("%y.%m.%d")
                postdates.append(postdate)
                
                #제목
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                titles.append(title)
                
                #블로거이름
                blogger_name = response_body_dict['items'][j]['bloggername']
                blogger_names.append(blogger_name)
                
    # utf-8형식으로 저장하면 엑셀에서 열때 에러나지만, load는 가능
    if save == True:
        crawling_df = pd.DataFrame({"post_dates":postdates, "title":titles, "blogger_name":blogger_names, "full_text":strings, "url":urls})
        crawling_df.to_csv(keyword+"_"+str(display_count* page_count)+".csv",encoding='utf-8',index= False)
    return crawling_df

In [5]:
crawling_df = get_blog_post(keyword, display_count, page_count, sort_type)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


### 2020-04-01 09:25:17  같은거 중복으로 들어오는 오류발생
* 확인 display_count는 표시되는 개수에 불과하며 page_count가 1개씩 넘어갈때마다 나머지 diplay_count가 중복됨  
이하 확인자료 페이지가 넘어가면서 1번째있는 블로그외 다른 블로그들은 유지됨 

In [21]:
response_code

200

In [10]:
response_bodys[0]

{'lastBuildDate': 'Wed, 01 Apr 2020 09:36:15 +0900',
 'total': 755029,
 'start': 1,
 'display': 5,
 'items': [{'title': '<b>양양</b>서핑 우와 뜨겁네!',
   'link': 'https://blog.naver.com/shakyamuni?Redirect=Log&logNo=221878085927',
   'description': '오늘은 <b>양양</b> 서핑 으로 동해에서 가장 핫한 서핑샵 플리즈웨잇서프를 소개해 드리려고 해요! <b>양양</b>은 강원도 내에서도 많이 유명한 지역이죠! 서퍼들이 많이 모이면서 동네 자체가 아주 힙하고 독특한... ',
   'bloggername': '피위의 행복한 어드벤쳐',
   'bloggerlink': 'https://blog.naver.com/shakyamuni',
   'postdate': '20200328'},
  {'title': '<b>양양</b> 설해원 마운틴스테이',
   'link': 'https://blog.naver.com/rockncat?Redirect=Log&logNo=221872610847',
   'description': '강원도 <b>양양</b>에 위치한 설해원이예요 <b>양양</b> 공항에서 2~3분 거리입니다 여기가 공항인지 설해원인지 할 정도로 그냥 바로 옆이라고 생각하시면 됩니다 체크인하러 먼저 들러야 하는 클럽 하우스 옆에 있는... ',
   'bloggername': '반짝반짝 냥이, 손!',
   'bloggerlink': 'https://blog.naver.com/rockncat',
   'postdate': '20200325'},
  {'title': '푸짐했던 <b>양양</b> 맛집',
   'link': 'https://blog.naver.com/collier13?Redirect=Log&logNo=221841451893',
   'description': '얼마전에 가족

In [13]:
response_bodys[1]

{'lastBuildDate': 'Wed, 01 Apr 2020 09:36:15 +0900',
 'total': 755029,
 'start': 2,
 'display': 5,
 'items': [{'title': '<b>양양</b> 설해원 마운틴스테이',
   'link': 'https://blog.naver.com/rockncat?Redirect=Log&logNo=221872610847',
   'description': '강원도 <b>양양</b>에 위치한 설해원이예요 <b>양양</b> 공항에서 2~3분 거리입니다 여기가 공항인지 설해원인지 할 정도로 그냥 바로 옆이라고 생각하시면 됩니다 체크인하러 먼저 들러야 하는 클럽 하우스 옆에 있는... ',
   'bloggername': '반짝반짝 냥이, 손!',
   'bloggerlink': 'https://blog.naver.com/rockncat',
   'postdate': '20200325'},
  {'title': '푸짐했던 <b>양양</b> 맛집',
   'link': 'https://blog.naver.com/collier13?Redirect=Log&logNo=221841451893',
   'description': '얼마전에 가족들과 <b>양양</b>에 놀러갔다가 아주 괜찮은 <b>양양</b> 맛집을 찿아갔어요~ 대게 전문점이었는데 신선함은 물론이고 푸짐하고 든든한 한끼를 제대로 먹고 올 수있어서 만족했던 곳이에요 제가 찿은 곳은... ',
   'bloggername': '브로컬리의세상',
   'bloggerlink': 'https://blog.naver.com/collier13',
   'postdate': '20200307'},
  {'title': '<b>양양</b> 쏠비치 리조트 2박3일 맛집투어',
   'link': 'https://blog.naver.com/ashacs1?Redirect=Log&logNo=221856482406',
   'description': '오랜만에 <b

In [14]:
response_bodys[2]

{'lastBuildDate': 'Wed, 01 Apr 2020 09:36:16 +0900',
 'total': 755029,
 'start': 3,
 'display': 5,
 'items': [{'title': '푸짐했던 <b>양양</b> 맛집',
   'link': 'https://blog.naver.com/collier13?Redirect=Log&logNo=221841451893',
   'description': '얼마전에 가족들과 <b>양양</b>에 놀러갔다가 아주 괜찮은 <b>양양</b> 맛집을 찿아갔어요~ 대게 전문점이었는데 신선함은 물론이고 푸짐하고 든든한 한끼를 제대로 먹고 올 수있어서 만족했던 곳이에요 제가 찿은 곳은... ',
   'bloggername': '브로컬리의세상',
   'bloggerlink': 'https://blog.naver.com/collier13',
   'postdate': '20200307'},
  {'title': '<b>양양</b> 쏠비치 리조트 2박3일 맛집투어',
   'link': 'https://blog.naver.com/ashacs1?Redirect=Log&logNo=221856482406',
   'description': '오랜만에 <b>양양</b> 쏠비치 다녀왔어요- 날씨가 아직도 춥기에 완전 무장을 하고 다녀왔어요- 가자마자... 덕분에 <b>양양</b> 시내 나갔다가 옷집이 없어서, 강릉까지 가서 운동화, 바지를 사고왔네요 ㅠㅠ 그래도... ',
   'bloggername': '[코리아에듀]▶️기업교육&컨설팅',
   'bloggerlink': 'https://blog.naver.com/ashacs1',
   'postdate': '20200316'},
  {'title': '<b>양양</b> 낙산비치호텔, 마운틴뷰 덕에 상쾌했던 숙소-♬',
   'link': 'https://blog.naver.com/eruriee?Redirect=Log&logNo=221874324050',
   'de

In [25]:
    encode_keyword = urllib.parse.quote(keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 페이지수까지 까지 각각 하나씩 긁어옴 
    for i in tqdm_notebook(range(1, 100 + 1),desc = "page work"):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        #접속오류났을때 종료후 저장
        try:
            response = urllib.request.urlopen(request)
            response_code = response.getcode()

            # respon결과가 200일때 결과읽어옴 
            if response_code is 200:
                response_body = response.read()
                response_body_dict = json.loads(response_body.decode('utf-8'))
                responses.append(response_body_dict)
        except :
            pass

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
response_body_dict

{'lastBuildDate': 'Wed, 01 Apr 2020 15:20:38 +0900',
 'total': 755097,
 'start': 100,
 'display': 1,
 'items': [{'title': '강원도 <b>양양</b> 바다보이는 숙소 소소한이야기 302호',
   'link': 'https://blog.naver.com/eun_churing?Redirect=Log&logNo=221879305813',
   'description': '#<b>양양</b>숙소 #강원도숙소 #바다보이는펜션 #소소한이야기 #강원도여행 &quot;강원도 <b>양양</b> 소소한이야기 펜션 &quot; 인스타 구경하다가 우연히 발견한 <b>양양</b>펜션 소소한이야기 바다 풍경이기도 하고 숙소 자체가 너무 이뻐서... ',
   'bloggername': 'churing',
   'bloggerlink': 'https://blog.naver.com/eun_churing',
   'postdate': '20200329'}]}

### 2020-04-01 13:41:21  http 5000오류
* 640번대정도에서 오류가 발생하는데 오류발생후 대처방안
-> 오류발생해도 진행하게끔 만듬

In [12]:
def get_blog_post(keyword, display_count, page_count, sort_type,save = True):
    '''
    keyword : 검색하고 싶은 키워드
    display_count : 한 페이지당 표출할 개수 min = 10, max = 100
    page_count : 총 진행할 페이지의 수 min = 1, max = 1000
    sort_type : 정렬옵션 "sim" (유사도순), "date" (날짜순)
    
    display_count는 표시되는 개수에 불과하며 page_count가 1개씩 넘어갈때마다 나머지 diplay_count가 중복됨
    ''' 
    #날짜와 본문내용 저장
    postdates = []
    strings = []
    urls = []
    titles = []
    blogger_names = []
    responses = []

    encode_keyword = urllib.parse.quote(keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 페이지수까지 까지 각각 하나씩 긁어옴 
    for i in tqdm_notebook(range(1, 100 + 1),desc = "page work"):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        #접속오류났을때 종료후 저장
        try:
            response = urllib.request.urlopen(request)
            response_code = response.getcode()

            # respon결과가 200일때 결과읽어옴 
            if response_code is 200:
                response_body = response.read()
                response_body_dict = json.loads(response_body.decode('utf-8'))
                responses.append(response_body_dict)
                try:
                #items의 개수만큼씩 진행
                    for j in range(0, len(response_body_dict['items'])):
                        blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

                        #블로그 url안에 들어가기(아직 크롤링불가)
                        get_blog_post_content_code = requests.get(blog_post_url)
                        get_blog_post_content_text = get_blog_post_content_code.text
                        get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
                        #크롤링가능한 url에 접속
                        real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text
                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')
                        #본문부분 추출 postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함 
                        blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

                        #포스트날짜
                        postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],"%Y%m%d").strftime("%y.%m.%d")
                        postdates.append(postdate)

                        #제목
                        remove_html_tag = re.compile('<.*?>')
                        title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                        titles.append(title)

                        #블로거이름
                        blogger_name = response_body_dict['items'][j]['bloggername']
                        blogger_names.append(blogger_name)

                        #전체 텍스트 
                        string = ""
                        for sentence in blog_post_content[0].stripped_strings:
                            string += " "+sentence.replace('\xa0'," ")
                        #비언어 텍스트제거 
                        string = del_outword(string)
                        strings.append([string]) 

                        #url
                        urls.append(real_blog_post_url)

                except Exception as ex:
                    print('추출에러 {num}번째'.format(num = i),ex)
                    pass
            else :
                pass

        except Exception as ex:
            print('open에러 {num}번째'.format(num = i),ex)
            pass
                
    # utf-8형식으로 저장하면 엑셀에서 열때 에러나지만, load는 가능
    if save == True:
        crawling_df = pd.DataFrame({"post_dates":postdates, "title":titles, "blogger_name":blogger_names, "full_text":strings, "url":urls})
        crawling_df.to_csv(keyword+"_"+str(display_count* page_count)+".csv",encoding='utf-8',index= False)
    return crawling_df

In [13]:
crawling_df = get_blog_post(keyword, display_count, page_count, sort_type)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


추출에러 145번째 list index out of range
추출에러 153번째 list index out of range
추출에러 183번째 list index out of range
추출에러 191번째 list index out of range
추출에러 197번째 list index out of range
추출에러 200번째 list index out of range
추출에러 213번째 list index out of range
추출에러 281번째 list index out of range
추출에러 322번째 list index out of range
추출에러 334번째 list index out of range
추출에러 367번째 list index out of range
추출에러 402번째 list index out of range
추출에러 408번째 list index out of range
추출에러 490번째 list index out of range
추출에러 494번째 list index out of range
추출에러 498번째 list index out of range
추출에러 560번째 list index out of range
추출에러 586번째 list index out of range
추출에러 590번째 list index out of range
추출에러 603번째 list index out of range
추출에러 606번째 list index out of range
추출에러 634번째 list index out of range
추출에러 641번째 list index out of range
추출에러 643번째 list index out of range
open에러 647번째 HTTP Error 500: Internal Server Error
open에러 650번째 HTTP Error 500: Internal Server Error
open에러 653번째 HTTP Error 500: Internal Server Error
open에러 

open에러 798번째 HTTP Error 500: Internal Server Error
open에러 799번째 HTTP Error 500: Internal Server Error
open에러 800번째 HTTP Error 500: Internal Server Error
open에러 801번째 HTTP Error 500: Internal Server Error
open에러 802번째 HTTP Error 500: Internal Server Error
open에러 803번째 HTTP Error 500: Internal Server Error
open에러 804번째 HTTP Error 500: Internal Server Error
open에러 805번째 HTTP Error 500: Internal Server Error
open에러 806번째 HTTP Error 500: Internal Server Error
open에러 807번째 HTTP Error 500: Internal Server Error
open에러 808번째 HTTP Error 500: Internal Server Error
open에러 809번째 HTTP Error 500: Internal Server Error
open에러 810번째 HTTP Error 500: Internal Server Error
open에러 811번째 HTTP Error 500: Internal Server Error
open에러 812번째 HTTP Error 500: Internal Server Error
open에러 813번째 HTTP Error 500: Internal Server Error
open에러 814번째 HTTP Error 500: Internal Server Error
open에러 815번째 HTTP Error 500: Internal Server Error
open에러 816번째 HTTP Error 500: Internal Server Error
open에러 817번째 HTTP Error 500: In

open에러 959번째 HTTP Error 500: Internal Server Error
open에러 960번째 HTTP Error 500: Internal Server Error
open에러 961번째 HTTP Error 500: Internal Server Error
open에러 962번째 HTTP Error 500: Internal Server Error
open에러 963번째 HTTP Error 500: Internal Server Error
open에러 964번째 HTTP Error 500: Internal Server Error
open에러 965번째 HTTP Error 500: Internal Server Error
open에러 966번째 HTTP Error 500: Internal Server Error
open에러 967번째 HTTP Error 500: Internal Server Error
open에러 968번째 HTTP Error 500: Internal Server Error
open에러 969번째 HTTP Error 500: Internal Server Error
open에러 970번째 HTTP Error 500: Internal Server Error
open에러 971번째 HTTP Error 500: Internal Server Error
open에러 972번째 HTTP Error 500: Internal Server Error
open에러 973번째 HTTP Error 500: Internal Server Error
open에러 974번째 HTTP Error 500: Internal Server Error
open에러 975번째 HTTP Error 500: Internal Server Error
open에러 976번째 HTTP Error 500: Internal Server Error
open에러 977번째 HTTP Error 500: Internal Server Error
open에러 978번째 HTTP Error 500: In

ValueError: arrays must all be same length

In [14]:
crawling_df.shape

NameError: name 'crawling_df' is not defined

### 연관분석

In [13]:
# crawling_df_s = pd.read_csv('양양_1000.csv',encoding='utf-8')

In [21]:
crawling_df_s[crawling_df_s['url'] == 'https://blog.naver.com/collier13?Redirect=Log&logNo=221841451893']

,post_dates,title,full_text,url
1,20.03.07,푸짐했던 양양 맛집,[' 얼마전에 가족들과 양양에 놀러갔다가 아주 괜찮은 양양 맛집을 찿아갔어요~ 대게...,https://blog.naver.com/collier13?Redirect=Log&...
10,20.03.07,푸짐했던 양양 맛집,[' 얼마전에 가족들과 양양에 놀러갔다가 아주 괜찮은 양양 맛집을 찿아갔어요~ 대게...,https://blog.naver.com/collier13?Redirect=Log&...


In [20]:
crawling_df_s['url'][1]

'https://blog.naver.com/collier13?Redirect=Log&logNo=221841451893'

In [211]:
komoran = Komoran(userdic='user_dictionary.txt')

#이모티콘 제거 [출처](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)

In [ ]:
                string = string.replace('\u200b'," ")
                string = string.replace('\n'," ")
                string = string.replace('😑','')
                strings.append([string]) 

In [220]:
sw = ['오늘','시간','최근','지난','관련','이번','이날','현재','10','올해','지난해','있다','이상','기준','때문',
     "메뉴","주문","양양","강원도",'정도','도착','생각','양양군','사진','위치','바다','해변']
all_nouns =[]

for i in tqdm_notebook(range(0,len(crawling_df))):
    try:
        nouns = komoran.nouns(crawling_df['full_text'][i])
    except :
        nouns = emoji_pattern.sub(r'',crawling_df['full_text'][399])
        nouns = han.sub(r'',TEXT_refine)
    clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),nouns))
    all_nouns.append(clean_nouns)

C:\Users\try00\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [221]:
noun_df = pd.DataFrame({'nouns':all_nouns})

In [222]:
rules = apriori(all_nouns, min_support=0.2, min_condience =0.3)

In [223]:
results  = list(rules)

In [224]:
results_df = pd.DataFrame(columns =["lhs",'rhs','support','confidence','lift'])
index =0
for row in results:   
    support = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = " ".join(x for x in ordered_item[0])
        rhs = " ".join(x for x in ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        results_df.loc[index] = [lhs,rhs,support,confidence,lift]
        index = index +1

In [228]:
results_df = results_df.sort_values(by="confidence",ascending=False)

In [229]:
results_df

,lhs,rhs,support,confidence,lift
26,,여행,0.423,0.423,1.0
20,,사람,0.410,0.410,1.0
35,,코로나,0.379,0.379,1.0
25,,아침,0.311,0.311,1.0
15,,마음,0.308,0.308,1.0
13,,느낌,0.292,0.292,1.0
16,,맛집,0.290,0.290,1.0
36,,풍경,0.288,0.288,1.0
10,,기분,0.279,0.279,1.0
19,,보니,0.275,0.275,1.0
